In [ ]:
from struct import pack
from struct import unpack

In [ ]:
command_codes = { 
    'STOP': 1,
    'PRINT': 2,
    'PRINT_INT': 3,
    'READ_INT': 4,
    'MV': 5,
    'JUMP': 6,
    'IF_EQ': 7,
    'CJUMP': 8,
    'ADD': 9,
    'SUB': 10,
    'PUSH': 11,
    'POP': 12,
    'GET': 13,
    'PUSH_IP': 14,
    'LS_INC': 15,
    'LS_DEC': 16
}

In [ ]:
def add_int(line, var_addresses, binary, position):
    name, value = line.split(' ')
    var_addresses[name] = position
    binary.write(pack('>I', int(value)))
    return position + 1

In [ ]:
def add_str(line, var_addresses, binary, position):
    name, value = line.split(' ', 1)
    var_addresses[name] = position
    value = value[1:-1]
    binary.write(pack('>I', len(value)))
    value += (4 - len(value) % 4) * '\0'
    binary.write(value)
    return position + len(value) / 4 + 1

In [ ]:
def position_to_address(position):
    return pack('B', position * 4)

In [ ]:
def position_from_address(position):
    return pack('>H', position * 4)

In [ ]:
def assembly_command(command, args, var_addresses, code_labels, binary, position, bad_labels):
    binary.write(pack('B', command_codes[command]))
    if command == 'STOP':
        binary.write(b'\0\0\0')
    elif command == 'PRINT':
        binary.write(b'\0')
        binary.write(position_from_address(var_addresses[args[0]]))
    elif command == 'PRINT_INT':
        binary.write(b'\0')
        binary.write(position_from_address(var_addresses[args[0]]))
    elif command == 'READ_INT':
        binary.write(b'\0\0\0')
    elif command == 'MV':
        binary.write(position_to_address(var_addresses[args[0]]))
        binary.write(position_from_address(var_addresses[args[1]]))
    elif command == 'IF_EQ':
        binary.write(position_to_address(var_addresses[args[0]]))
        binary.write(position_from_address(var_addresses[args[1]]))
    elif command == 'CJUMP' or command == 'JUMP':
        binary.write(b'\0')
        if args[0] in code_labels:
            binary.write(position_from_address(code_labels[args[0]]))
        else:
            bad_labels[args[0]] = [position, 0]
            binary.write(b'\0\0')
    elif command == 'ADD':
        binary.write(position_to_address(var_addresses[args[0]]))
        binary.write(position_from_address(var_addresses[args[1]]))
    elif command == 'SUB':
        binary.write(position_to_address(var_addresses[args[0]]))
        binary.write(position_from_address(var_addresses[args[1]]))
    elif command == 'PUSH':
        binary.write(b'\0')
        binary.write(position_from_address(var_addresses[args[0]]))
    elif command == 'POP':
        binary.write(b'\0')
        binary.write(position_from_address(var_addresses[args[0]]))
    elif command == 'GET':
        binary.write(position_to_address(var_addresses[args[0]]))
        binary.write(position_from_address(var_addresses[args[1]]))

    return position + 1

In [ ]:
def parse_code_line(line):
    splitted = line.split('"')
    if splitted[0] == '':
        label = splitted[1]
        splitted = splitted[2][1:].split(' ', 1)
        return label, splitted[0], splitted[1].split(' ') if len(splitted) > 1 else None
    else:
        splitted = line.split(' ')
        return None, splitted[0], splitted[1:] if len(splitted) > 1 else None

In [ ]:
def assembly_data(lines, binary):
    var_addresses = {}
    position = 0
    position = add_int('IP 0', var_addresses, binary, position)
    position = add_int('LL 0', var_addresses, binary, position)
    position = add_int('LS 0', var_addresses, binary, position)
    position = add_int('_TMP_RET 0', var_addresses, binary, position)
    for line in lines:
        if line.find('"') == -1:
            position = add_int(line, var_addresses, binary, position)
        else:
            position = add_str(line, var_addresses, binary, position)
    return var_addresses, position

In [ ]:
def assembly_code(parsed_commands, var_addresses, binary, position):
    code_labels = {}
    bad_labels = {}
    start_position = position
    for parsed_command in parsed_commands:
        label, command, args = parsed_command
        if label is not None:
            code_labels[label] = position
            if label in bad_labels:
                bad_labels[label][1] = position
        position = assembly_command(command, args, var_addresses, code_labels, binary, position, bad_labels)
    return bad_labels, start_position, position

In [ ]:
def prepare_asm(path_to_asm):
    lines = open(path_to_asm, 'r')
    constants = {}
    data_lines = []
    code_lines = []
    functions_lines = []
    additional_var_id = 0
    for line in lines:
        if line.find("START") != -1:
            break
        data_lines.append(line[:-1])
    for line in lines:
        if line.find("STACK") != -1:
            break
        parsed_command = parse_code_line(line[:-1]) 
        if parsed_command[1] == "CALL":
            code_lines.append([None, 'PUSH', [parsed_command[2][1]]])
            code_lines.append([None, 'PUSH_IP', ['2']])
            code_lines.append([None, 'JUMP', [parsed_command[2][0]]])
        elif parsed_command[1] == "RET":
            code_lines.append([None, 'POP', ['_TMP_RET']])
            code_lines.append([None, 'LS_DEC', ['LS']])
            code_lines.append([None, 'LS_DEC', ['1']])
            code_lines.append([None, 'PUSH', [parsed_command[2][0]]])
            code_lines.append([None, 'JUMP', ['_TMP_RET']])
    new_parsed_commands = []
    for command in code_lines:
        new_declarations, new_command = replace_consts(command, constants)
        for new_declaration in new_declarations:
            data_lines.append[new_declarations]
        new_parsed_commands.append(new_command)
    lines.close()
    return data_lines, new_parsed_commands

In [ ]:
def assembly(path_to_asm, path_to_save):
    data_lines, parsed_commands = prepare_asm(path_to_asm)
    binary = open(path_to_save, 'w+b')
    
    var_addresses, position = assembly_data(data_lines, binary)
    bad_labels, start_position, stack_position = assembly_code(parsed_commands, var_addresses, binary, position)
    binary.write(b'\0'*1024)
    binary.seek(0)
    binary.write(pack('>I', start_position * 4))
    binary.seek(8)
    binary.write(pack('>I', stack_position * 4))
    for key, value in bad_labels.iteritems():
        binary.seek(value[0] * 4 + 2)
        binary.write(position_from_address(value[1]))
    binary.close()

In [ ]:
import mmap
block_size = 4
IP_position = 0
LL_position = block_size
LS_position = block_size * 2

In [ ]:
def STOP(IP, args, binary):
    return True

In [ ]:
def PRINT(IP, args, binary):
    length = get_int(args[1], binary)
    start = args[1]+block_size
    print binary[start:start+length]
    set_int(IP_position, binary, IP + block_size)
    return False

In [ ]:
def PRINT_INT(IP, args, binary):
    print get_int(args[1], binary)
    set_int(IP_position, binary, IP + block_size)
    return False

In [ ]:
def READ_INT(IP, args, binary):
    number = int(input())
    addr = get_int(LS_position, binary) + block_size
    set_int(LS_position, binary,  addr)
    set_int(addr, binary, number)
    set_int(IP_position, binary, IP + block_size)
    return False

In [ ]:
def MV(IP, args, binary):
    binary[args[0]:args[0]+block_size] = binary[args[1]:args[1]+block_size]
    set_int(IP_position, binary, IP + block_size)
    return False

In [ ]:
def JUMP(IP, args, binary):
    set_int(IP_position, binary, args[1])
    return False

In [ ]:
def IF_EQ(IP, args, binary):
    set_int(LL_position, binary, get_int(args[0], binary) == get_int(args[1], binary))
    set_int(IP_position, binary, IP + block_size)
    return False

In [ ]:
def CJUMP(IP, args, binary):
    if get_int(LL_position, binary):
        return JUMP(IP, args, binary)
    set_int(IP_position, binary, IP + block_size)
    return False

In [ ]:
def ADD(IP, args, binary):
    set_int(args[0], binary, get_int(args[0], binary) + get_int(args[1], binary))
    set_int(IP_position, binary, IP + block_size)
    return False

In [ ]:
def SUB(IP, args, binary):
    set_int(args[0], binary, get_int(args[0], binary) - get_int(args[1], binary))
    set_int(IP_position, binary, IP + block_size)
    return False

In [ ]:
def PUSH(IP, args, binary):
    addr = get_int(LS_position, binary) + block_size
    set_int(LS_position, binary, addr)
    binary[addr:addr+block_size] = binary[args[1]:args[1]+block_size]
    set_int(IP_position, binary, IP + block_size)
    return False

In [ ]:
def POP(IP, args, binary):
    addr = get_int(LS_position, binary)
    binary[args[1]:args[1]+block_size] = binary[addr:addr+block_size]
    set_int(LS_position, binary, addr - block_size)
    set_int(IP_position, binary, IP + block_size)
    return False

In [ ]:
def GET(IP, args, binary):
    offset = get_int(args[0], binary) * 4
    addr = get_int(LS_position, binary)
    binary[args[1]:args[1]+block_size] = binary[addr-offset:addr-offset+block_size]
    set_int(IP_position, binary, IP + block_size)
    return False

In [ ]:
def PUSH_IP(IP, args, binary):
    offset = get_int(args[1], binary) * 4
    addr = get_int(LS_position, binary) + block_size
    set_int(LS_position, binary, addr)
    set_int(binary[addr:addr+block_size], binary, get_int(IP_position, binary) + offset)
    set_int(IP_position, binary, IP + block_size)
    return False

In [ ]:
def LS_INC(IP, args, binary):
    offset = get_int(args[1], binary) * 4
    addr = get_int(LS_position, binary)
    set_int(LS_position, binary, addr + offset)
    set_int(IP_position, binary, IP + block_size)
    return False

In [ ]:
def LS_DEC(IP, args, binary):
    offset = get_int(args[1], binary) * 4
    addr = get_int(LS_position, binary)
    set_int(LS_position, binary, addr - offset)
    set_int(IP_position, binary, IP + block_size)
    return False

In [ ]:
functions = { 
    1: STOP,
    2: PRINT,
    3: PRINT_INT,
    4: READ_INT,
    5: MV,
    6: JUMP,
    7: IF_EQ,
    8: CJUMP,
    9: ADD,
    10: SUB,
    11: PUSH,
    12: POP,
    13: GET,
    14: PUSH_IP,
    15: LS_INC,
    16: LS_DEC
}

In [ ]:
def get_int(position, binary):
    return unpack('>I', binary[position:position + block_size])[0]

In [ ]:
def set_int(position, binary, value):
    binary[position:position + block_size] = pack('>I', value)

In [ ]:
def get_command(position, binary):
    return unpack('B', binary[position])[0]

In [ ]:
def get_args(position, binary):
    return unpack('B', binary[position+1])[0], unpack('>H', binary[position+2:position+4])[0]

In [ ]:
def run(path_to_bin):
    f = open(path_to_bin, 'rb')
    binary = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_COPY)
    stopped = False
    while not stopped:
        IP = get_int(IP_position, binary)
        function = functions[get_command(IP, binary)]
        args = get_args(IP, binary)
#         print 'current LS', get_int(LS_position, binary), get_command(IP, binary), args
        stopped = function(IP, args, binary)

In [ ]:
binary = assembly('fib.asm', 'fib.bin')

In [ ]:
run('fib.bin')